# Comparing HiCAT E2E vs. matrix PASTIS contrast

In [ ]:
from astropy.io import fits
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from config import CONFIG_PASTIS
from e2e_simulators.hicat_imaging import set_up_hicat
import util as util

In [ ]:
nb_seg = CONFIG_PASTIS.getint('HiCAT', 'nb_subapertures')

In [ ]:
# HiCAT direct PSF and normalization
hicat = set_up_hicat(apply_continuous_dm_maps=True)
hicat.include_fpm = False
data = hicat.calc_psf()
psf = data[0].data
norm = psf.max()
print(norm)

plt.imshow(np.log(psf))

In [ ]:
# HiCAT coronagraph floor
hicat = set_up_hicat(apply_continuous_dm_maps=True)
hicat.include_fpm = True
data = hicat.calc_psf()
coro = data[0].data / norm

iwa = CONFIG_PASTIS.getfloat('HiCAT', 'IWA')
owa = CONFIG_PASTIS.getfloat('HiCAT', 'OWA')
sampling = CONFIG_PASTIS.getfloat('HiCAT', 'sampling')

dh_mask = util.create_dark_hole(coro, iwa=iwa, owa=owa, samp=sampling)
coro_floor = util.dh_mean(coro, dh_mask)
print(f'Coro floor: {coro_floor}')

In [ ]:
# Load the matrix
matrix = fits.getdata('/Users/ilaginja/data_from_repos/pastis_data/2020-08-26T11-44-28_hicat/matrix_numerical/PASTISmatrix_num_piston_Noll1.fits')
print(matrix.shape)

In [ ]:
plt.imshow(matrix)

## Create aberation

In [ ]:
# Create random aberrations normalized to a certain WFE rms value
rms_norm = 2 *u.nm
print(rms_norm)

In [ ]:
aber = np.random.random([nb_seg]) 
rms_init = util.rms(aber)
aber *= rms_norm.value / rms_init
aber *= u.nm
print(aber)

In [ ]:
aber -= np.mean(aber)
print(aber)

## E2E contrast

In [ ]:
# Put aberration on Iris AO and calculate PSF
plt.figure(figsize=(12, 10))
for nseg in range(nb_seg):
    hicat.iris_dm.set_actuator(nseg, aber[nseg], 0, 0)

psf_hicat = hicat.calc_psf(display=True, return_intermediates=False)
psf_hicat = psf_hicat[0].data / norm

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.log10(psf_hicat))
plt.colorbar()

In [ ]:
contrast_e2e = util.dh_mean(psf_hicat, dh_mask)
print(rms_norm)
print(contrast_e2e)

## PASTIS contrast

In [ ]:
contrast_pastis = util.pastis_contrast(aber, matrix) + coro_floor
print(contrast_pastis)

In [ ]:
print(matrix)